https://python.langchain.com/docs/how_to/custom_tools/

In [2]:
from getpass import getpass

In [3]:
OPENAI_API_KEY = getpass(prompt='Enter your OpenAI Token:')

In [4]:
from ase.units import kJ

In [5]:
from ase.build import bulk

In [6]:
from ase.calculators.emt import EMT
from ase.eos import calculate_eos

In [7]:
from ase.data import reference_states, atomic_numbers

In [8]:
from langchain_core.tools import tool

In [9]:
@tool
def get_crystal_structure(chemical_symbol: str) -> str:
    """Returns the atomic crystal structure of a chemcial symbol"""
    ref_state = reference_states[atomic_numbers[chemical_symbol]]
    if ref_state is None:
        return "No crystal structure known."
    else:
        return ref_state["symmetry"]

In [10]:
# Includes a try except workflow, that returns an error message to the LLM agent, but it can continue 

@tool
def get_bulk_modulus(chemical_symbol: str, crystal_structure: str) -> str:
    """Returns the bulk modulus of chemcial symbol for a given crystal structure in GPa"""
    try:
        atoms = bulk(name=chemical_symbol, crystalstructure=crystal_structure,)
        atoms.calc = EMT()
        eos = calculate_eos(atoms)
        v, e, B = eos.fit()
        return B / kJ * 1.0e24
    except Exception as error:
        # handle the exception
        print("An exception occurred:", error)

In [11]:
el = "Al"
crystal_structure = get_crystal_structure.invoke("Al")
get_bulk_modulus.invoke({"chemical_symbol": "Al", "crystal_structure": crystal_structure})

39.91626202223279

In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

In [13]:
tools = [get_crystal_structure, get_bulk_modulus]

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # "You are very powerful assistant, but don't know current events.",
            "You are very powerful assistant, but don't know current events. For each query vailidate that it contains a chemical element and otherwise cancel.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [15]:
llm_with_tools = llm.bind_tools(tools)

In [16]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [17]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## We can also ask inverse questions

### Specify a set of elements 

In [18]:

lst = list(agent_executor.stream({"input": "I would like to have a material with a bulk modulus of around 170GPa. Is Aluminium, Silver or Gold better? "})) # This works well



> Entering new AgentExecutor chain...

Invoking: `get_bulk_modulus` with `{'chemical_symbol': 'Al', 'crystal_structure': 'fcc'}`


39.91626202223279
Invoking: `get_bulk_modulus` with `{'chemical_symbol': 'Ag', 'crystal_structure': 'fcc'}`


100.15712339997654
Invoking: `get_bulk_modulus` with `{'chemical_symbol': 'Au', 'crystal_structure': 'fcc'}`


173.8389836420435The bulk modulus for Aluminum (Al) is approximately 39.92 GPa, for Silver (Ag) is approximately 100.16 GPa, and for Gold (Au) is approximately 173.84 GPa. 

Based on the bulk modulus requirement of around 170 GPa, Gold (Au) would be the best choice as it has a bulk modulus closest to the desired value.

> Finished chain.


### Specify maximum number of simulations to answer the question (n=10)
Since LLMs are probabilistic, the results (elements selected) will vary with repeated execution

In [25]:
# We see that it selects the right one from the 10 examples but Gold was not part of the 10 simulations. 
lst = list(agent_executor.stream({"input": "I would like to have a material with a bulk modulus of around 170GPa. Is there a transition metal from the periodic system that satisifies this? Try maxium 10 chemical elements. Please output a list at the end of your calculations with all of the values calculated."}))  





> Entering new AgentExecutor chain...

Invoking: `get_crystal_structure` with `{'chemical_symbol': 'V'}`


bcc
Invoking: `get_crystal_structure` with `{'chemical_symbol': 'Nb'}`


bcc
Invoking: `get_crystal_structure` with `{'chemical_symbol': 'Ta'}`


bcc
Invoking: `get_crystal_structure` with `{'chemical_symbol': 'Mo'}`


bcc
Invoking: `get_crystal_structure` with `{'chemical_symbol': 'W'}`


bcc
Invoking: `get_crystal_structure` with `{'chemical_symbol': 'Tc'}`


hcp
Invoking: `get_crystal_structure` with `{'chemical_symbol': 'Re'}`


hcp
Invoking: `get_crystal_structure` with `{'chemical_symbol': 'Ru'}`


hcp
Invoking: `get_crystal_structure` with `{'chemical_symbol': 'Rh'}`


fcc
Invoking: `get_crystal_structure` with `{'chemical_symbol': 'Pd'}`


fcc
Invoking: `get_bulk_modulus` with `{'chemical_symbol': 'V', 'crystal_structure': 'bcc'}`


An exception occurred: No EMT-potential for V
None
Invoking: `get_bulk_modulus` with `{'chemical_symbol': 'Nb', 'crystal_structure': 'bcc'}`

# Lessons learned: 
* With Error handling we can enable multiple simulations and inverse questions. The reasoning, comparing multiple simulation outputs works well